In [1]:
import tushare as ts
import sina_data
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from datetime import datetime, timedelta
from dateutil.parser import parse
import time
import common_util
import os

In [2]:
def get_time(date=False, utc=False, msl=3):
    if date:
        time_fmt = "%Y-%m-%d %H:%M:%S.%f"
    else:
        time_fmt = "%H:%M:%S.%f"

    if utc:
        return datetime.utcnow().strftime(time_fmt)[:(msl-6)]
    else:
        return datetime.now().strftime(time_fmt)[:(msl-6)]


def print_info(status="I"):
    return "\033[0;33;1m[{} {}]\033[0m".format(status, get_time())

In [3]:
def judgement(df, change_rate=0.01, buy1_rate=0.03, buy1_volume=1e5):
    float_share = df['float_share'].to_numpy().astype(np.int)
    open = df['今日开盘价'].to_numpy().astype(np.float)
    pre_close = df['昨日收盘价'].to_numpy().astype(np.float)
    limit_up = limit_up_price(pre_close)
    price = df['当前价'].to_numpy().astype(np.float)
    high = df['今日最高价'].to_numpy().astype(np.float)
    low = df['今日最低价'].to_numpy().astype(np.float)
    volume = df['成交股票数'].to_numpy().astype(np.int)
    buy_1v = df['买一量'].to_numpy().astype(np.int)

    judge_list = [
        low < limit_up,
        price < limit_up,
        volume < float_share * change_rate,
        buy_1v < float_share * buy1_rate,
        buy_1v < buy1_volume
    ]

    return judge_list


# 基于前一交易日收盘价的涨停价计算
def limit_up_price(pre_close):
    return np.around(pre_close * 1.1, decimals=2)


# 日K数据判断是否开板
def is_sold(code, start_date):
    print(code)
    try:
        time.sleep(1)
        pro = ts.pro_api('ba73b3943bdd57c2ff05991f7556ef417f457ac453355972ff5d01ce')
        start_date = (parse(str(start_date))+timedelta(1)).strftime('%Y%m%d')
        end_date = datetime.now().strftime('%Y%m%d')
        daily_k = pro.daily(ts_code=code, start_date=start_date, end_date=end_date)
        if len(daily_k) > 0:
            daily_k['flag'] = daily_k.apply(
                lambda x: x['high'] == x['low'] and x['open'] == x['close'] and x['high'] == x['low'],
                axis=1
            )
            flag = daily_k['flag'].sum()
            result = True
            for each in daily_k['flag'].tolist():
                result = result and each
            return result
        else:
            return True
    except Exception as e:
        print('再次请求ts数据')
        time.sleep(1)
        a = is_sold(code, start_date)
        return a


# 获取流通股本
def get_float_share(code):
    print(code)
    try:
        time.sleep(1)
        pro = ts.pro_api('ba73b3943bdd57c2ff05991f7556ef417f457ac453355972ff5d01ce')
        # target_date = datetime.now().strftime('%Y%m%d')
        target_data = []
        delta = 0
        count = 1
        while len(target_data) == 0:
            target_date = datetime.now() + timedelta(delta)
            target_data = pro.daily_basic(
                ts_code=code, trade_date=target_date.strftime('%Y%m%d'), fields='free_share'
            )
            delta = delta - 1
            time.sleep(0.5)
            count = count + 1
            if count > 3:
                return 1000000
        return target_data.values[0][0] * 10000

    except Exception as e:
        time.sleep(1)
        get_float_share(code)
        print('再次请求ts数据.....')

In [4]:
# 新股筛选
# 获取股票列表
pro = ts.pro_api('ba73b3943bdd57c2ff05991f7556ef417f457ac453355972ff5d01ce')
basic_data = pro.stock_basic()
print('股票筛选')
# basic_data.to_excel(r'C:\Users\duanp\Desktop\test\stock_basic.xlsx')
# basic_data = pd.read_excel(r'C:\Users\duanp\Desktop\test\stock_basic.xlsx')
# 筛选上市日期为近一月的股票
start_date = datetime.now() + timedelta(-30)
end_date = datetime.now() + timedelta(1)
basic_data['list_date'] = basic_data['list_date'].apply(lambda x: parse(str(x)))
basic_data = basic_data[basic_data['list_date'] > start_date]
basic_data = basic_data[basic_data['list_date'] < end_date]
# 剔除科创板股票
basic_data = basic_data[basic_data['market'] != '科创板']
# 筛选未开板的股票
basic_data['target_flag'] = basic_data.apply(lambda x: is_sold(x['ts_code'], x['list_date']), axis=1)
# basic_data = basic_data[basic_data['target_flag']]
print('补充流通股本数据')
# 补充流通股本信息
basic_data['float_share'] = basic_data.apply(lambda x: get_float_share(x['ts_code']), axis=1)
basic_data['float_share'] = basic_data['float_share'].fillna('100000')
print('预警股票如下：')
print(basic_data)

change_rate = 0.01
buy1_rate = 0.03
buy1_volume = 1e5

tick_list = [
    '股票代码',
    '今日开盘价',
    '昨日收盘价',
    '当前价',
    '今日最高价',
    '今日最低价',
    '成交股票数',
    '买一量'
]

flag_dict = {
    "low_flag": "当日曾开板！",
    "price_flag": "已经开板！",
    "volume_top_flag": "换手率超过 {:.0%}！".format(change_rate),
    "buy1_percent_flag": "买一量不足总流通市值的 {:.0%}！".format(buy1_rate),
    "buy1_volume_flag": "买一量不足 {} 股！".format(buy1_volume),
}

flag_list = list(flag_dict.keys())
flag_len = len(flag_list)

股票筛选
003001.SZ
003012.SZ
003013.SZ
003015.SZ
003016.SZ
003017.SZ
300898.SZ
300899.SZ
300900.SZ
300901.SZ
300902.SZ
300999.SZ
601187.SH
601568.SH
601995.SH
605058.SH
605169.SH
605336.SH
605338.SH
补充流通股本数据
003001.SZ
003012.SZ
003013.SZ
003015.SZ
003016.SZ
003017.SZ
300898.SZ
300899.SZ
300900.SZ
300901.SZ
300902.SZ
300999.SZ
601187.SH
601568.SH
601995.SH
605058.SH
605169.SH
605336.SH
605338.SH
预警股票如下：
        ts_code  symbol  name area industry market  list_date  target_flag  \
1417  003001.SZ  003001  中岩大地   北京     建筑工程    中小板 2020-10-13        False   
1427  003012.SZ  003012  东鹏控股   广东       陶瓷    中小板 2020-10-19        False   
1428  003013.SZ  003013  地铁设计   广东     建筑工程    中小板 2020-10-22        False   
1429  003015.SZ  003015  日久光电   江苏      元器件    中小板 2020-10-21        False   
1430  003016.SZ  003016  欣贺股份   福建       服饰    中小板 2020-10-26        False   
1431  003017.SZ  003017  大洋生物   浙江     化工原料    中小板 2020-10-26        False   
2296  300898.SZ  300898  熊猫乳品   浙江      乳制品    创业板 2

In [5]:
basic_data['target_code'] = basic_data['ts_code'].apply(lambda x: common_util.get_format_code(x, 'num'))

In [7]:
basic_data['ts_code'].to_list()

['003001.SZ',
 '003012.SZ',
 '003013.SZ',
 '003015.SZ',
 '003016.SZ',
 '003017.SZ',
 '300898.SZ',
 '300899.SZ',
 '300900.SZ',
 '300901.SZ',
 '300902.SZ',
 '300999.SZ',
 '601187.SH',
 '601568.SH',
 '601995.SH',
 '605058.SH',
 '605169.SH',
 '605336.SH',
 '605338.SH']

In [8]:
tick_data = sina_data.get_tick_data(basic_data['symbol'].to_list())

In [9]:
tick_data['股票代码'] = tick_data['股票代码'].apply(lambda x: common_util.get_format_code(x, 'wind'))

In [10]:
tick_data = tick_data[tick_list]

In [11]:
temp_data = basic_data.merge(tick_data, left_on='ts_code', right_on='股票代码')

In [12]:
judge_list = judgement(temp_data, change_rate, buy1_rate, buy1_volume)

In [13]:
judge_list

[array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]),
 array([ True,  True,  True, False,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]),
 array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False]),
 array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]),
 array([ True,  True,  True, False,  True,  True,  True,  True,  True,
        False,  True, False, False,  True, False,  True,  True,  True,
         True])]

In [14]:
alert_dict = dict()
count = 0

In [34]:
for idx in range(flag_len):
    temp_data[flag_list[idx]] = judge_list[idx]
    alert_dict[flag_list[idx]] = temp_data[temp_data[flag_list[idx]]]["name"].tolist()
    if len(alert_dict[flag_list[idx]]) > 0:
        print(print_info("W"), end=" ")
        print(flag_dict[flag_list[idx]])
        print("，".join(alert_dict[flag_list[idx]]))
    else:
        count += 1

[W 23:24:45.211] 当日曾开板！
中岩大地，东鹏控股，地铁设计，日久光电，欣贺股份，大洋生物，熊猫乳品，上海凯鑫，C广联，C中胤，C国安达，金龙鱼，厦门银行，北元集团，中金公司，澳弘电子，N洪通，帅丰电器，巴比食品
[W 23:24:45.211] 已经开板！
中岩大地，东鹏控股，地铁设计，欣贺股份，大洋生物，熊猫乳品，上海凯鑫，C广联，C中胤，C国安达，金龙鱼，厦门银行，北元集团，中金公司，澳弘电子，N洪通，帅丰电器，巴比食品
[W 23:24:45.212] 买一量不足总流通市值的 3%！
中岩大地，东鹏控股，地铁设计，日久光电，欣贺股份，大洋生物，熊猫乳品，上海凯鑫，C广联，C中胤，C国安达，金龙鱼，厦门银行，北元集团，中金公司，澳弘电子，N洪通，帅丰电器，巴比食品
[W 23:24:45.213] 买一量不足 100000.0 股！
中岩大地，东鹏控股，地铁设计，欣贺股份，大洋生物，熊猫乳品，上海凯鑫，C广联，C国安达，北元集团，澳弘电子，N洪通，帅丰电器，巴比食品


In [23]:
idx=1
temp_data[flag_list[idx]] = judge_list[idx]

In [26]:
alert_dict[flag_list[idx]] = temp_data[temp_data[flag_list[idx]]]
alert_dict

{'low_flag':       ts_code  symbol  name area industry market  list_date  target_flag  \
 0   003001.SZ  003001  中岩大地   北京     建筑工程    中小板 2020-10-13        False   
 1   003012.SZ  003012  东鹏控股   广东       陶瓷    中小板 2020-10-19        False   
 2   003013.SZ  003013  地铁设计   广东     建筑工程    中小板 2020-10-22        False   
 3   003015.SZ  003015  日久光电   江苏      元器件    中小板 2020-10-21        False   
 4   003016.SZ  003016  欣贺股份   福建       服饰    中小板 2020-10-26        False   
 5   003017.SZ  003017  大洋生物   浙江     化工原料    中小板 2020-10-26        False   
 6   300898.SZ  300898  熊猫乳品   浙江      乳制品    创业板 2020-10-16        False   
 7   300899.SZ  300899  上海凯鑫   上海     环境保护    创业板 2020-10-16        False   
 8   300900.SZ  300900   C广联  黑龙江       航空    创业板 2020-10-29        False   
 9   300901.SZ  300901   C中胤   浙江       服饰    创业板 2020-10-29        False   
 10  300902.SZ  300902  C国安达   福建     专用机械    创业板 2020-10-29        False   
 11  300999.SZ  300999   金龙鱼   上海       食品    创业板 2020-10-15    